In [1]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import tables
from random import shuffle
from IPython.display import clear_output
from sklearn import metrics
from tqdm import tqdm
import time
%matplotlib inline

import os
USE_GPU = True
os.environ['CUDA_VISIBLE_DEVICES'] = '3' if USE_GPU else ''
EPS = 1e-15

/opt/conda/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# for phase 1 set num_classes=3
# for phase 2 set num_classes=4
num_classes = 4

energy_cut = 0.0099999985 #0.01
zero_class_exists = False

num_nn_output = num_classes - (not zero_class_exists)

In [3]:
train_file = '/home/leyla/data/train_5-6.csv'
#train_file2 = '/home/leyla/data/train_3-4.csv'
test_file = '/home/leyla/data/test_5-6.csv'
submission_file = '/home/leyla/data/submission_simplified_5-6.csv'

In [4]:
#a = pd.read_csv(train_file1) 
#b = pd.read_csv(train_file2) 

In [5]:
#b['event'] += 5000

In [6]:
#b = b.dropna()

In [7]:
#merged = a.append(b)
#merged.to_csv('/home/leyla/data/output.csv', index = False)

In [8]:
#train_file = '/home/leyla/data/output.csv'

In [9]:
#tr = pd.read_csv(train_file)
#tr.head()

In [10]:
%load_ext autoreload
%autoreload 2

# Prepare data

In [11]:
import sys
sys.path.append("/home/leyla/baseline_mpnn/") 

In [12]:
from tools.base import plot_3d, hdf5_to_numpy, plot_3d_with_edges, csv_to_numpy, X_MAX, Y_MAX, Z_MAX
from tools.tools import stretch_array

In [13]:
import psutil

process = psutil.Process(os.getpid())

In [14]:
process.memory_info().rss / 2**30

0.1245269775390625

In [15]:
%%time
X, Y, M, _ = csv_to_numpy(file=train_file, num_classes=num_classes, 
                               zero_class_exists=zero_class_exists, energy_cut=energy_cut)

CPU times: user 21.5 s, sys: 4.36 s, total: 25.8 s
Wall time: 24.3 s


In [16]:
#k = -2
#plot_3d(X[k], Y[k])

# Creation of graph dataset

In [17]:
n_neighbors = 80
mode = 'kneighbors_graph' #'radius_neighbors_graph' 

in_degree_max, out_degree_max = 0, 0

In [18]:
from tools.simplified_clustering import generate_graph_dataset

In [19]:
X_clusters_graph = []
lx = len(X)
for k in tqdm(range(lx)):
    if len(X[k]) == 0:
        continue
    # construction of graph based on aggregated statistics
    X_cluster_graph, in_degree_max_local, out_degree_max_local = generate_graph_dataset(X=X[k], Y=Y[k], M=M[k],
                                                                                       n_neighbors=n_neighbors,
                                                                                       mode = mode
                                                                                       )
    in_degree_max = max(in_degree_max_local, in_degree_max)
    out_degree_max = max(out_degree_max_local, out_degree_max)
    
    X_clusters_graph.append(X_cluster_graph)

100%|██████████| 5000/5000 [04:54<00:00, 16.99it/s]


In [20]:
#k = 0
#plot_3d_with_edges(X[k], Y[k], X_clusters_graph[k]['X_cluster_in_out'])

In [21]:
in_degree_max, out_degree_max

(25, 8)

In [22]:
# padding
for X_cluster_graph in X_clusters_graph:
    X_cluster_graph['X_cluster_messages_out'] = stretch_array(X_cluster_graph['X_cluster_messages_out'], 
                                                              n=out_degree_max, 
                                                              fill_value=len(X_cluster_graph['X_cluster_edges']))
    
    X_cluster_graph['X_cluster_messages_in'] = stretch_array(X_cluster_graph['X_cluster_messages_in'], 
                                                              n=in_degree_max, 
                                                              fill_value=len(X_cluster_graph['X_cluster_edges']))

# Deep learning model (MPNN)




In [23]:
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda, GRU
from keras.optimizers import RMSprop
from keras import backend as K
import keras
import tensorflow as tf
from keras.layers import Dropout
from keras.models import Sequential
from keras.activations import relu

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

/opt/conda/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Placeholders

In [24]:
X_cluster_graph['X_cluster_nodes'].shape, X_cluster_graph['X_cluster_edges'].shape

((2099, 1), (405638, 5))

__X_nodes__ -- features per hit(i.e. energy).

__X_edges__ -- features for each edge that connects two hits(i.e. relative difference of coordinates).

__X_labels__ -- labels ;)

In [39]:
ndim_features_nodes = 4
ndim_features_edges = 5 
ndim_message = 6

X_nodes = K.placeholder(shape=(None, ndim_features_nodes)) # features of nodes
X_edges = K.placeholder(shape=(None, ndim_features_edges)) # features of edges
X_labels = K.placeholder(shape=(None, num_nn_output)) # labels

__X_nodes_in_out__ -- edge list.

__X_messages_in__ -- in-adjacency lists.

__X_messages_out__ -- out-adjacency lists.

All these graph representations are equivalent. 

In [40]:
X_nodes_in_out = K.placeholder(shape=(None, 2), dtype=np.int32) # edges
X_messages_in = K.placeholder(shape=(None, in_degree_max), dtype=np.int32) # shape = (none, size of neighbourhood)
X_messages_out = K.placeholder(shape=(None, out_degree_max), dtype=np.int32) # shape = (none, size of neighbourhood)

# fake messages to(or from) non-existing node
fake_message_const = K.constant(value=[ndim_message * [-np.inf]]) 

In [41]:
placeholders = {
    'X_nodes': X_nodes,
    'X_edges': X_edges,
    'X_labels': X_labels,
    'X_nodes_in_out': X_nodes_in_out,
    'X_messages_in': X_messages_in,
    'X_messages_out': X_messages_out
}

In [42]:
m = 8 #2

### NNs

In [43]:
steps = 6

message_passers = {
    0: Sequential(layers=[
                      Dense(64, input_shape=(2 * ndim_features_nodes + ndim_features_edges,), activation=relu), 
                     # Dropout(rate=0.05),
                      Dense(ndim_message, activation=relu), #,
                    #  Dropout(rate=0.05),
                  ]
                 ),
    1: Sequential(layers=[
                      Dense(64, input_shape=(2 * ndim_features_nodes + ndim_features_edges,), activation=relu), 
                    #  Dropout(rate=0.05),
                      Dense(ndim_message, activation=relu), #,
                    #  Dropout(rate=0.05),
                  ]
                 ),    
    2: Sequential(layers=[
                      Dense(64, input_shape=(2 * ndim_features_nodes + ndim_features_edges,), activation=relu), 
                     # Dropout(rate=0.05),
                      Dense(ndim_message, activation=relu) ,
                    #  Dropout(rate=0.05),
                  ]
                 ),
    3: Sequential(layers=[
                      Dense(64, input_shape=(2 * ndim_features_nodes + ndim_features_edges,), activation=relu), 
                    #  Dropout(rate=0.05),
                      Dense(ndim_message, activation=relu) ,
                    #  Dropout(rate=0.05),
                  ]
                 ),
    4: Sequential(layers=[
                      Dense(64, input_shape=(2 * ndim_features_nodes + ndim_features_edges,), activation=relu), 
                    #  Dropout(rate=0.05),
                      Dense(ndim_message, activation=relu) ,
                   #   Dropout(rate=0.05),
                  ]
                 ),
     5: Sequential(layers=[
                      Dense(64, input_shape=(2 * ndim_features_nodes + ndim_features_edges,), activation=relu), 
                    #  Dropout(rate=0.05),
                      Dense(ndim_message, activation=relu) ,
                   #   Dropout(rate=0.05),
                  ]
                 )
}

In [44]:
#state_updater = tf.contrib.rnn.LSTMCell(num_units=ndim_features_nodes)
#state_updater = tf.contrib.rnn.GRUCell(num_units=ndim_features_nodes)
state_updater = Sequential(layers=[
                     Dense(32, input_shape=(m * ndim_message + ndim_features_nodes,), activation=relu), 
                     Dense(ndim_features_nodes)
                                 ]
                         )

In [45]:
#readout = Dense(num_nn_output, input_shape=(ndim_features_nodes,), activation=keras.activations.softmax)

readout = Sequential(layers=[
                     Dense(64, input_shape=(ndim_features_nodes,), activation=relu),
                     Dense(num_nn_output, activation=keras.activations.softmax)
                                 ]
                         )

### MPNN construction

In [46]:
from tools.mpnn import build_network, run_train, run_test

In [47]:
X_predictions = build_network(X_nodes=X_nodes, 
                              X_edges=X_edges, 
                              X_nodes_in_out=X_nodes_in_out, 
                              X_messages_in=X_messages_in, 
                              X_messages_out=X_messages_out, 
                              message_passers=message_passers, 
                              state_updater=state_updater, 
                              readout=readout, 
                              steps=steps, 
                              ndim_features_nodes=ndim_features_nodes,
                              fake_message_const=fake_message_const)

In [48]:
loss_tf = tf.reduce_mean(keras.losses.categorical_crossentropy(X_labels, X_predictions))
accuracy_tf = tf.reduce_mean(keras.metrics.categorical_accuracy(X_labels, X_predictions))

In [49]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss_tf, var_list=tf.trainable_variables())

/opt/conda/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [50]:
sess = tf.Session()
init = tf.global_variables_initializer()
init.run(session=sess)

## Split dataset

In [51]:
TRAIN_SIZE = int(len(X_clusters_graph) * 0.9)
print(TRAIN_SIZE)
shuffle(X_clusters_graph)

X_clusters_graph_train = X_clusters_graph[:TRAIN_SIZE]
X_clusters_graph_eval = X_clusters_graph[TRAIN_SIZE:]

900


### Train

In [52]:
losses = []
accuracies = []
roc_aucs = []
n = 15

for epoch in tqdm(range(n)):
    loss_float = 0
    accuracy_float = 0
    
    losses_epoch = []
    accuracies_epoch = []
    roc_aucs_epoch = []
    for X_cluster_graph in X_clusters_graph_train:
        predictions, (loss, accuracy) = run_train(X_cluster_graph=X_cluster_graph,
                                   X_predictions=X_predictions,
                                   optimizer=optimizer, sess=sess, 
                                   ndim_features_nodes=ndim_features_nodes, 
                                   ndim_features_edges=ndim_features_edges, 
                                   placeholders=placeholders,
                                   metrics=[loss_tf, accuracy_tf])
        losses_epoch.append(loss)
        accuracies_epoch.append(accuracy)
    clear_output()
    
    losses.append(np.mean(losses_epoch))
    plt.title('log-loss')
    plt.plot(losses)
    plt.show()

    accuracies.append(np.mean(accuracies_epoch))
    plt.title('accuracy')
    plt.plot(accuracies)
    plt.show()


  0%|          | 0/15 [00:00<?, ?it/s]

ResourceExhaustedError: OOM when allocating tensor with shape[4995,1258,6] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: sub_46 = Sub[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](GatherV2_101, Mean_116)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: Mean_123/_353 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_4220_Mean_123", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'sub_46', defined at:
  File "/opt/conda/anaconda3/lib/python3.6/runpy.py", line

## Eval

In [56]:
losses_test = [] 
accuracies_test = [] 
roc_aucs_test = [] 
predictions_total = [] 
y_total =[]

for X_cluster_graph in X_clusters_graph_eval: 
    predictions = run_test(X_cluster_graph=X_cluster_graph, 
                           X_predictions=X_predictions, 
                           sess=sess, 
                           ndim_features_nodes=ndim_features_nodes, 
                           ndim_features_edges=ndim_features_edges, 
                           placeholders=placeholders) 
    X_cluster_graph['predictions'] = predictions 
    predictions_total.append(predictions) 
    y_total.append(X_cluster_graph['Y_cluster_labels']) 
    losses_test.append(loss) 
    accuracies_test.append(accuracy)

In [57]:
#losses_test = []
#accuracies_test = []
#roc_aucs_test = []
#predictions_total = [] 
#y_total =[]

#for X_cluster_graph in X_clusters_graph_eval:
 #   predictions, (loss, accuracy) = run_test(X_cluster_graph=X_cluster_graph, 
  #                                            X_predictions=X_predictions,
   #                                           sess=sess, 
    #                                          ndim_features_nodes=ndim_features_nodes, 
     #                                         ndim_features_edges=ndim_features_edges, 
      #                                        placeholders=placeholders,
       #                                       metrics=[loss_tf, accuracy_tf])
 #   X_cluster_graph['predictions'] = predictions
  #  predictions_total.append(predictions)
   # y_total.append(X_cluster_graph['Y_cluster_labels'])
    #losses_test.append(loss)
    #accuracies_test.append(accuracy)

In [58]:
predictions_total = np.concatenate(predictions_total)
y_total = np.concatenate(y_total)

In [59]:
from sklearn import metrics
accuracy = metrics.accuracy_score(np.argmax(y_total, axis=1), np.argmax(predictions_total, axis=1))
roc_auc = metrics.roc_auc_score(y_total, predictions_total)

In [60]:
accuracy 
# start -0.8683611126422625
# use gru - 0.8687567396308694
# neib = 30 - 0.8910923820501319
# neib = 40 - 0.9082779642419363
# neib = 45 - 0.9158992339666193
# epoch = 20 - 0.9206353632299216
# epoch = 25 neib = 50 - 0.9264679126592279
# LSTM - 0.9237462378528039
# data 14xxx - 0.9292922456499576
# data 14xxx neib =30 0.9140251065371974 
# data 5000 neib = 30 layers = 15 - 0.9060545732807426
# neib - 60 + k_mean - 0.9299395850343464
# energy 0.095 neib 20 ep 10 k_mean- 0.9069938532462544
# energy 0.095 neib 60 ep 20 k_mean 0.9315556814741793 bad bad bad
# fec 8 0.932470521438493 
#fec 12 0.9401650911552497 
# dataset 14xxx 0.9396454244919843  # ds 5xxx 0.9437540667624275 # 0.9413805860665672 #0.9496136479567739 
#0.9505675006817211
#0.9568560565133952

0.9568560565133952

In [61]:
roc_auc 
# 0.9666363977125328
# 0.9633168628170781
# 0.973642993604225
# 0.9809168220142711
# 0.9833718768518708
# 0.9848123191935816
# 0.98685898792743
# 0.9861728000922451
# 0.9876960468540009
# 0.9882631114162708
# 0.9882631114162708
# 0.9892055238144396
# 0.9825319454860798
# 0.9898732963738727
# 0.9895222142393605  
#0.990904425606832
#0.9919523564090716
# 0.9925245011296729
#0.9931252926811585 #0.9936682934108406 #0.9950754738272782

0.9950754738272782

## Test

In [62]:
test_file

'/home/leyla/data/test_5-6.csv'

In [63]:
X_test, Y_test, M_test, events_test = csv_to_numpy(file=test_file, num_classes=num_classes, test=True,
                                                   zero_class_exists=zero_class_exists, energy_cut=energy_cut)
shift = 0 if zero_class_exists else 1

In [64]:
import tables
preds_array = []

for k, event in tqdm(list(enumerate(events_test))):
        X_cluster_graph, in_degree_max_local, out_degree_max_local = generate_graph_dataset(X=X_test[k], 
                                                                                            Y=Y_test[k], 
                                                                                            M=M_test[k],
                                                                                            n_neighbors=n_neighbors, 
                                                                                            in_degree_max=in_degree_max, 
                                                                                            out_degree_max=out_degree_max)
        
        predictions = run_test(X_cluster_graph=X_cluster_graph, 
                                               X_predictions=X_predictions,
                                               sess=sess, 
                                               ndim_features_nodes=ndim_features_nodes, 
                                               ndim_features_edges=ndim_features_edges, 
                                               placeholders=placeholders)
        preds_array.append(pd.DataFrame.from_dict(
            {
                'x': (X_test[k][:, 0] * X_MAX).astype(int), 
                'y': (X_test[k][:, 1] * Y_MAX).astype(int), 
                'z': (X_test[k][:, 2] * Z_MAX).astype(int),
                'event': event, 
                'pred': np.argmax(predictions, axis=1) + shift,
        }))


100%|██████████| 5000/5000 [24:11<00:00,  3.44it/s]


In [65]:
preds_df = pd.concat(preds_array)  

preds_df.to_csv(submission_file, index=False)
#preds_array.close()
#f_submission.close()

In [66]:
process.memory_info().rss / 2**30

54.11070251464844